# Importing Packages

In [2]:
import sys
import pandas as pd
import math
import numpy as np

import matplotlib
from matplotlib import pyplot as plt
from matplotlib import style
import seaborn as sns
import tradingeconomics as te
from datetime import datetime,timedelta
import os
import requests

In [3]:
pd.options.display.max_columns=1000

# Exploration

In [77]:
#df=pd.read_excel('REACH_RCA_Base_de_donnees_Suivi_des_marches_Mai_2023.xlsx', sheet_name='Données nettoyées', header=1)
#df=pd.read_excel('REACH_RCA_Base_de_donnees_Suivi_des_marches_Avril_2023.xlsx', sheet_name='Données nettoyées', header=1)
#df=pd.read_excel('REACH_RCA_Base_de_donnees_Suivi_des_marches_Mars_2023.xlsx', sheet_name='Données nettoyées', header=1)
#df=pd.read_excel('REACH_RCA_Base_de_donnees_Suivi_des_marches_Fevrier_2023.xlsx', sheet_name='Données nettoyées', header=1)
#df=pd.read_excel('REACH_RCA_Base_de_donnees_Suivi_des_marches_Janvier_2023-1.xlsx', sheet_name='Données nettoyées', header=1)
#df=pd.read_excel('REACH_RCA_Base_de_donnees_Suivi_des_marches_Novembre_2022.xlsx', sheet_name='Données nettoyées', header=1)
#df=pd.read_excel('REACH_RCA_Base_de_donnees_Suivi_des_marches_Octobre_2022.xlsx', sheet_name='Données nettoyées', header=1)
#df=pd.read_excel('REACH_RCA_Base_de_donnees_Suivi_des_marches_Septembre_2022.xlsx', sheet_name='Données nettoyées', header=1)
#df=pd.read_excel('REACH_RCA_Base_de_donnees_Suivi_des_marches_Aout_2022.xlsx', sheet_name='Données nettoyées', header=1)
#df=pd.read_excel('REACH_RCA_Base_de_donnees_Suivi_des_marches_Juillet_2022.xlsx', sheet_name='Données nettoyées', header=1)
#df=pd.read_excel('REACH_RCA_Base_de_donnees_Suivi_des_marches_Juin_2022.xlsx', sheet_name='Données nettoyées', header=1)
#df=pd.read_excel('REACH_RCA_Base_de_donnees_Suivi_des_marches_Mai_2022.xlsx', sheet_name='Données nettoyées', header=1)
df=pd.read_excel('REACH_RCA_Base_de_donnees_Suivi_des_marches_Aout_2023-1.xlsx', sheet_name='Prix médians', header=3)

In [8]:
df_m=df[['uuid', 'start', 'end', 'today', 'diviceid', 'q0_3_date', 'q0_4_admin1',
       'q0_5_admin2', 'q0_6_admin3', 'q0_7_localite', 'q0_8_market_size']]

In [7]:
df.rename(columns={'deviceid':'diviceid'}, inplace=True)

In [127]:
#df.loc[0, 'q1_articles_habituels']
#, 'version__'

In [9]:
C=df.loc[:, 'q1_articles_habituels.nfi_moustiquaire':].drop(['commentaire', 'id', 'submission_time', 'validation_status', 'notes', 'status', 'submitted_by', 'version__', 'tags', 'index'], axis=1).columns.tolist()

In [10]:
K=[]
for j in C:
    if 'approvisionnement' in j:
        K.append(j)

In [11]:
#df['q2_1_4_moustiquaire_approvisionnement'].nunique()

In [12]:
K_=[]
for j in K:
    if df[j].nunique()==2 and ('oui' in df[j].unique().tolist()) and ('non' in df[j].unique().tolist()):
        K_.append(j)

In [13]:
K__=[]
for j in K:
    if j.endswith('raison'):
        K__.append(j)

In [14]:
#df_m=pd.concat([df_m, df[K_]], axis=1)
df_=pd.DataFrame()
df_1=pd.DataFrame()
for j in K_:
    df_=pd.concat([df_, df_m[df[j]=='oui']])
    df_['product']=j[7:-18]
    df_1=pd.concat([df_1, df_])

In [15]:
df_22=df_1.reset_index().drop('index', axis=1)
for i in range(len(df_22)):
    id=df_22.loc[i, 'uuid']
    p=df_22.loc[i, 'product']
    for j in K__:
        if p in j:
            df_22.loc[i, 'supply constraints types']=df[df['uuid']==id][j].tolist()[0]

In [16]:
df_22['supply constraints']=1

In [17]:
#df_m=pd.concat([df_m, df[K_]], axis=1)
df_=pd.DataFrame()
df_1=pd.DataFrame()
for j in K_:
    df_=pd.concat([df_, df_m[df[j]=='non']])
    df_['product']=j[7:-18]
    df_1=pd.concat([df_1, df_])

In [18]:
df_22b=df_1.copy()
df_22b['supply constraints']=0
df_22b['supply constraints types']=np.nan

In [19]:
df_2=pd.DataFrame()
df_2=pd.concat([df_2, df_22, df_22b])

In [ ]:
df_2

In [21]:
#df_n=pd.read_excel('REACH_App_new.xlsx')
#df_2_=pd.concat([df_n, df_2])

In [22]:
df_2_.to_excel('REACH_App_new_.xlsx', index=False)

In [4]:
df_3=pd.read_excel('REACH_App.xlsx')

In [29]:
df_2_=df_2_.reset_index().drop('index', axis=1)

In [59]:
dff=df_2_.copy()
t=['trop_cher', 'etat_route', 'insecurite', 'absence_transport']
for j in range(len(t)):
    k=t[j]
    dff[k]=np.nan
    for i in range(len(df_2_)):
        if k in str(df_2_.loc[i, 'supply constraints types']):
            dff.loc[i, k]=1
        else:
            dff.loc[i, k]=0
#df_2_[df_2_['supply constraints types']=='trop_cher']

In [61]:
dff.to_excel('CK_approv.xlsx', index=False)

In [64]:
#df.loc[:, 'q1_articles_habituels.nfi_moustiquaire':]

In [24]:
for i in range(len(df_m)):
    #q=0
    t=[]
    for j in C:
        if str(df.loc[i,j])!='nan':
            #q+=df.loc[i,j]
            t.append(df.loc[i,j])
        else:
            t.append('nan')
    #if q==1:
        #k=t.index(1)
        #df_m.loc[i, 'Attribute']=C[k]

In [74]:
import unidecode

In [ ]:
df.rename(columns=lambda x: x.strip(), inplace=True)
id=[]
for i in range(len(df)):
    #if str(df.loc[i,'Préfectures'])=='nan':
        #df.loc[i,'Préfectures']=df.loc[i-1,'Préfectures']
    
    if  'TOTAL' in str(df.loc[i,'Préfectures']):
        id.append(i)

df=df[df.index < min(id)]
df['Marchés']=df['Marchés'].apply(lambda x: unidecode.unidecode(x))
#df['Préfectures']=df['Préfectures'].apply(lambda x: unidecode.unidecode(x))
df['Date']=pd.to_datetime('2022-08-01')
df=df[df.columns.tolist()[-1:] + df.columns.tolist()[:-1]]

In [83]:
df['Préfecturess']=np.nan

In [ ]:
for i in range(len(df)):
    try:
        k=df.loc[i, 'Marchés']
        df.loc[i, 'Préfecturess']=df_[df_['marchés']==k]['préfectures'].tolist()[0]
    except:
        pass

df.insert(loc=1, column='Préfectures', value=df.Préfecturess.tolist())
df.drop('Préfecturess', axis=1, inplace=True)

In [86]:
df.columns=df.columns.str.lower()

In [107]:
C=['moustiquaire', 'bidon', 'drap',
       'pagne', 'natte', 'bâche', 'marmite', 'cuvette métallique',
       'maïs en grains', 'manioc cossette', 'riz', 'haricot', 'arachide',
       'viande', 'huile végétale', 'sucre', 'sel', 'savon', 'théière / bouta',
       'seau plastique', 'bois de chauffage', 'essence', 'eau bidon 20l']

In [109]:
#df_.columns=df_.columns.str.lower()
#dfff=df[['date', 'préfectures', 'marchés']].reset_index().drop('index', axis=1)
#df=df.reset_index().drop('index', axis=1)
dfff_=pd.DataFrame()
for i in C:
    dfff['product']=i
    dfff['values']=np.nan
    for j in range(len(df)):
        dfff.loc[j, 'values']=df.loc[j, i]
    
    dff_1=dfff.copy()
    
    dfff_=pd.concat([dfff_, dff_1])

In [92]:
df=pd.concat([d, df])

In [94]:
df.to_excel('REACH_Append_prix_median.xlsx', index=False)

In [12]:
df.reset_index().drop('index', axis=1).to_excel('REACH_Append_May.xlsx', index=False)

In [2]:
df=pd.read_excel('REACH_Append_May.xlsx')

In [ ]:
"Bossangoa"-----"Ouham"
'Mbaiki'-----'Lobaye'
'Dimbi'---'Basse-Kotto'

'Kouango'---'Ouaka'
'Kabo'---'Ouham'

In [120]:
#dfff_=dfff_.reset_index().drop('index', axis=1)
dfff_[dfff_['préfectures'].isnull()]

,date,préfectures,marchés,product,values


In [ ]:
dfff_.to_excel('REACH_Append_prices.xlsx', index=False)

In [119]:
for i in range(len(dfff_)):
    if dfff_.loc[i, 'marchés']=='Dimbi':
        dfff_.loc[i, 'préfectures']='Basse-Kotto'
    
    if dfff_.loc[i, 'marchés']=='Mobaye':
        dfff_.loc[i, 'préfectures']='Basse-Kotto'

    if dfff_.loc[i, 'marchés']=='Kouango':
        dfff_.loc[i, 'préfectures']='Ouaka'

    if dfff_.loc[i, 'marchés']=='Kabo':
        dfff_.loc[i, 'préfectures']='Ouham-Fafa'

    if dfff_.loc[i, 'marchés']=='Markounda':
        dfff_.loc[i, 'préfectures']='Ouham'
    
    if dfff_.loc[i, 'marchés']=='Koui':
        dfff_.loc[i, 'préfectures']='Ouham-Pendé'
    
    if dfff_.loc[i, 'marchés']=='Paoua':
        dfff_.loc[i, 'préfectures']='Lim-Pendé'
    
    #if df.loc[i, 'marchés']=='Mbaiki':
        #df.loc[i, 'préfectures']='Lobaye'

In [11]:
df[df['préfectures'].isnull()]

,date,préfectures,marchés,moustiquaire,bidon,drap,pagne,natte,bâche,marmite,...,viande,huile végétale,sucre,sel,savon,théière / bouta,seau plastique,bois de chauffage,essence,eau bidon 20l
